In [1]:
#one day i may move the functionality to start and monitor servers into this document, 
# but for now, start up everything with helao_interface.py before using this code
import requests
import json
from matplotlib import pyplot
url = "http://192.168.31.114:13380"
#url = "http://127.0.0.1:13380"
import time
import ipywidgets
import numpy
import pickle
import asyncio
import websockets
sub_data = {} #holds data sent to the notebook by the orchestrator
runs = {} #i think this holds the boolean variables that keep the websockets running?

In [2]:
#live data routines
#loop is websocket connection to receive data from the orchestrator
#update plot live plots this data as it comes in
#init_history_plot and load_figure are subroutines of update_plot
#much of this is adapted from https://stackoverflow.com/questions/68918341/live-update-several-plots-in-jupyter-notebook


uri = "ws://192.168.31.114:13380/ws_data/"

async def loop(ident): # non-blocking coroutine, updates data source
    if ident not in sub_data.keys():
        sub_data[ident] = []
    runs[ident] = True
    async with websockets.connect(uri+str(ident)) as ws:
        while runs[ident]:
            new_data = await ws.recv()
            new_data = json.loads(new_data)
            if None not in new_data:
                sub_data[ident].append(new_data)
                update_plot(ident)

def update_plot(ident):
    with plot_wrapper:
        fig, ax = load_figure(template_fig)
        plot_wrapper.clear_output(wait=True)
        l = len(sub_data[ident])
        for i in range(l):
            dat = sub_data[ident][i]
            dat = numpy.array(dat)
            ax.plot(dat[0],dat[1],str(1-(i+1)/l))
        pyplot.show()
        
def init_history_plot():
    """
    Create plot template (dump)
    
    Returns: pickled str
    """
    fig, ax = pyplot.subplots()
    dat = pickle.dumps(fig)
    pyplot.close()
    return dat


def load_figure(dump):
    """
    Load Figure from dump
    
    Returns: (Figure, Axes)
    """
    # https://github.com/ipython/ipykernel/issues/231
    import ipykernel.pylab.backend_inline as back_inline
    import matplotlib.backends.backend_agg as back_agg
    back_inline.new_figure_manager_given_figure = back_agg.new_figure_manager_given_figure
    figure = pickle.loads(dump)
    # https://github.com/matplotlib/matplotlib/issues/17627/
    figure._cachedRenderer = None
    return figure, figure.axes[0]

In [9]:
#start
#send this command to initialize the orchestrator for normal single experiments
collectionkey = 'default'
experiment = {'soe':['orchestrator/start'],'params':{'start':{'collectionkey':collectionkey}},'meta':{}}
requests.post(url+'/orchestrator/addExperiment',params = {'experiment':json.dumps(experiment)})

<Response [200]>

In [ ]:
#finish
#send this command to shut down the orchestrator for those
experiment = {'soe':['orchestrator/finish'],'params':{'finish':None},'meta':{}}
requests.post(url+'/orchestrator/addExperiment',params = {'experiment':json.dumps(experiment)})

In [161]:
#emergency functions
thread = 0
requests.post(url+'/orchestrator/clear',params = {'thread':thread})
#requests.post(url+'/orchestrator/kill',params = {'thread':thread})
#requests.post(url+'/orchestrator/pause',params = {'thread':thread})
#requests.post(url+'/orchestrator/resume',params = {'thread':thread})

<Response [200]>

In [15]:
#move the motors and/or take a measurement
probe = 'raman785' #probe used
sample = 'base' #sample type used -- determines coordinate system relative to probe
x = None #x to move to
y = None #y to move to; will not move if x and y both None
z = 17 #z height to move to; will not move if None
safe_z = None #safe z to move to after measurement; will not move if None
t = 60 #time to measure for, or number of scans if FTIR not on time mode
bg = None #use background for measurement
time_mode = True #toggle time-mode for ftir
experiment = {'soe':[],'params':{},'meta':{}}
if x != None or y != None:
    experiment['soe'].append('owis/movetable')
    experiment['params']['movetable'] = {'pos':json.dumps([x,y]),'probe':probe,'sample':sample}
if z != None:
    experiment['soe'].append('owis/moveprobe_1')
    experiment['params']['moveprobe_1'] = {'z':z,'probe':probe,'sample':sample}
if (probe == 'ftirLow' or probe == 'ftirHigh') and t != 0:
    experiment['soe'].append('arcoptix/read')
    experiment['params']['read'] = {'probe':probe,'time':time_mode,'av':t,'bg':bg}
if (probe == 'raman532' or probe == 'raman785') and t != 0:
    experiment['soe'].append('ocean/read')
    experiment['params']['read'] = {'t':t,'bg':bg}
if safe_z != None:
    experiment['soe'].append('owis/moveprobe_2')
    experiment['params']['moveprobe_2'] = {'z':z,'probe':probe,'sample':sample}


requests.post(url+'/orchestrator/addExperiment',params = {'experiment':json.dumps(experiment)})

<Response [200]>

In [7]:
#get data and start a live plotting instance
#this sets up a websocket connection with the orchestrator and begins live plotting of received spectra
%matplotlib inline
params = {'thread':0,'addresses':json.dumps(['read/data/res/wavenumbers','read/data/res/intensities'])}
ident = json.loads(requests.get(url+'/orchestrator/subscribe',params=params).json())['subscription_id']
asyncio.create_task(loop(ident))
plot_wrapper = ipywidgets.Output()
template_fig = init_history_plot()
plot_interface = ipywidgets.Box(children=[plot_wrapper])
plot_interface

Box(children=(Output(),))

In [147]:
#ftir queries
gain = None
experiment = {'soe':['arcoptix/saturation'],'params':{},'meta':{}}
requests.post(url+'/orchestrator/addExperiment',params = {'experiment':json.dumps({'soe':['arcoptix/saturation'],'params':{'saturation':{}},'meta':{}})})
#requests.post(url+'/orchestrator/addExperiment',params = {'experiment':json.dumps({'soe':['arcoptix/getGain'],'params':{'getGain':{}},'meta':{}})})
#requests.post(url+'/orchestrator/addExperiment',params = {'experiment':json.dumps({'soe':['arcoptix/setGain'],'params':{'setGain':{'gain':gain},'meta':{}})})
time.sleep(1)
sub_data[ident_arco]

[[0.05889075994491577, None]]

In [145]:
#maybe i also want a subscription to receive information about gain and saturation of the FTIR
async def arco_loop(ident): # non-blocking coroutine, updates data source
    if ident not in sub_data.keys():
        sub_data[ident] = []
    runs[ident] = True
    async with websockets.connect(uri+str(ident)) as ws:
        while runs[ident]:
            new_data = await ws.recv()
            new_data = json.loads(new_data)
            sub_data[ident].append(new_data)

params = {'thread':0,'addresses':json.dumps(['saturation/data/data/saturation','getGain/data/data/gain'])}
ident_arco = json.loads(requests.get(url+'/orchestrator/subscribe',params=params).json())['subscription_id']
asyncio.create_task(arco_loop(ident_arco))

<Task pending name='Task-50' coro=<arco_loop() running at C:\Users\JKFLOW~1\AppData\Local\Temp/ipykernel_12196/3681287511.py:1>>

In [76]:
#close a websocket on the orchestrator side (probably i need to use runs here also to shut it down on this side)
requests.get(url+'/orchestrator/unsubscribe',params={'ident':ident})

<Response [200]>

In [3]:
#can use orchestrator/getArray to request all spectra from a single run.
dorta = json.loads(requests.get(url+'/orchestrator/getArray',params={
    'addresses':json.dumps(['read/data/res/wavenumbers','read/data/res/intensities']),
    'path':'C:\\Users\\Operator\\Documents\\data\\z_cal\\hits_z_cal_session_31.h5','run':0}).json())

In [ ]:
#set raman background (out of date)
t = 5
experiment = {'soe':['owis/goref','ocean/readBackground'],
              'params':{'goref':{'probe':'raman','ref':'air'},
                        'readBackground':{'t':t}},
              'meta':{}}
requests.post(url+'/orchestrator/addExperiment',params = {'experiment':json.dumps(experiment)})


In [ ]:
#set ftir background (out of date)
t = 100
time = False
probe = 'ftirLow'
experiment = {'soe':['owis/goref','arcoptix/readBackground','owis/moveprobe'],
              'params':{'goref':{'probe':probe,'ref':'au'},
                        'readBackground':{'probe':probe,'time':time,'av':t},
                        'moveprobe':{'z':30,'probe':probe,'sample':'base'}},
              'meta':{}}
requests.post(url+'/orchestrator/addExperiment',params = {'experiment':json.dumps(experiment)})

In [173]:
#fix motor
motor = 0
experiment = {'soe':['owis/activate','owis/configure'],
              'params':{'activate':{'motor':motor},'configure':{'motor':motor}},'meta':{}}
requests.post(url+'/orchestrator/addExperiment',params = {'experiment':json.dumps(experiment)})

<Response [200]>

Task exception was never retrieved
future: <Task finished name='Task-27' coro=<loop() done, defined at C:\Users\JKFLOW~1\AppData\Local\Temp/ipykernel_10156/2452441317.py:3> exception=ConnectionClosedError(None, None, None)>
Traceback (most recent call last):
  File "c:\users\jkflowers\appdata\local\programs\python\python39\lib\site-packages\websockets\legacy\protocol.py", line 945, in transfer_data
    message = await self.read_message()
  File "c:\users\jkflowers\appdata\local\programs\python\python39\lib\site-packages\websockets\legacy\protocol.py", line 1015, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
  File "c:\users\jkflowers\appdata\local\programs\python\python39\lib\site-packages\websockets\legacy\protocol.py", line 1090, in read_data_frame
    frame = await self.read_frame(max_size)
  File "c:\users\jkflowers\appdata\local\programs\python\python39\lib\site-packages\websockets\legacy\protocol.py", line 1145, in read_frame
    frame = await Fra

In [ ]:
#default plotting method for single spectra
def plot(data):
    pyplot.plot(data[0],data[1])
    pyplot.show()

In [ ]:
#map on alexey's substrate
#larger experimental runs can be placed in their own cells like this
import csv
x0 = 7
y0 = -2
z = -3
t = 100
bg = True
time = False
sample = 'sem'
#start
collectionkey = 'alexey'
experiment = {'soe':['orchestrator/start'],'params':{'start':{'collectionkey':collectionkey}},'meta':{}}
requests.post(url+'/orchestrator/addExperiment',params = {'experiment':json.dumps(experiment)})
with open('C:/Users/jkflowers/Downloads/coord5x5.csv', newline='') as csvfile:
    reader = iter(csv.reader(csvfile, delimiter=','))
    next(reader)
    for line in reader:
        x,y = float(line[1])+x0,float(line[2])+y0
        experiment = {'soe':['owis/movetable','owis/moveprobe_1'],
                      'params':{'movetable':{'pos':json.dumps([x,y]),'probe':'ftir','sample':sample},
                                'moveprobe_1':{'z':z,'probe':'ftir','sample':sample}},
                      'meta':{}}
        requests.post(url+'/orchestrator/addExperiment',params = {'experiment':json.dumps(experiment)})
with open('C:/Users/jkflowers/Downloads/coord5x5_v2.csv', newline='') as csvfile:
    reader = iter(csv.reader(csvfile, delimiter=','))
    next(reader)
    for line in reader:
        x,y = float(line[1])+x0,float(line[2])+y0
        experiment = {'soe':['owis/movetable','owis/moveprobe_1'],
                      'params':{'movetable':{'pos':json.dumps([x,y]),'probe':'ftir','sample':sample},
                                'moveprobe_1':{'z':z,'probe':'ftir','sample':sample}},
                      'meta':{}}
        requests.post(url+'/orchestrator/addExperiment',params = {'experiment':json.dumps(experiment)})
with open('C:/Users/jkflowers/Downloads/coord5x5_v3.csv', newline='') as csvfile:
    reader = iter(csv.reader(csvfile, delimiter=','))
    next(reader)
    for line in reader:
        x,y = float(line[1])+x0,float(line[2])+y0
        experiment = {'soe':['owis/movetable','owis/moveprobe_1'],
                      'params':{'movetable':{'pos':json.dumps([x,y]),'probe':'ftir','sample':sample},
                                'moveprobe_1':{'z':z,'probe':'ftir','sample':sample}},
                      'meta':{}}
        requests.post(url+'/orchestrator/addExperiment',params = {'experiment':json.dumps(experiment)})
#finish
experiment = {'soe':['orchestrator/finish'],'params':{'finish':None},'meta':{}}
requests.post(url+'/orchestrator/addExperiment',params = {'experiment':json.dumps(experiment)})